# SETUP CELLS

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import requests
import json

# === Setup Paths ===
sys.path.insert(0, os.path.abspath("./genie-worksheets/src"))
root_dir = os.getcwd()

# === Load Environment ===
env_file = os.path.join(root_dir, ".env")
load_dotenv(env_file)

# === Verify API Key ===
AGENT_KEY = os.getenv("LLM_API_KEY")
BASE_URL = os.getenv("LLM_API_BASE_URL")
DATABASE_KEY = os.getenv("EXA_API_KEY")

# print(AGENT_KEY)


if not AGENT_KEY or AGENT_KEY == "your_api_key_here":
    raise ValueError("❌ Please set your LLM_API_KEY in .env file")

print("✓ Environment loaded")
print(f"✓ API Base URL: {BASE_URL}")


# === Import only what you need from Genie ===
from worksheets.specification.from_spreadsheet import gsheet_to_classes
print("✓ Genie Worksheets imported")

✓ Environment loaded
✓ API Base URL: https://api.openai.com/v1
✓ Genie Worksheets imported


In [2]:
# Ensure Environment is actually working
import requests
import json
from dotenv import load_dotenv

response = requests.post(
    f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {AGENT_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "user", "content": "Say hi in one word"}
        ],
        "max_tokens": 5
    }
)

if response.status_code == 200:
    result = response.json()
    print("Success!")
    print(f"Response: {result['choices'][0]['message']['content']}")
    print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
else:
    print(f"Error {response.status_code}: {response.text}")

Success!
Response: Hello!
Tokens used: 14


In [3]:
current_dir = os.getcwd()
root_dir = os.path.join(current_dir, "genie-worksheets")

# Credentials for reading the spreadsheet
!curl -L -o creds.zip "https://drive.google.com/uc?export=download&id=11QvSs2JZ5qpPrCvbX66Dg8pxfM_B8GaH"
!unzip creds.zip -d genie-worksheets/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3009  100  3009    0     0   2080      0  0:00:01  0:00:01 --:--:--  2563
Archive:  creds.zip
  inflating: genie-worksheets/worksheets/token.json  
  inflating: genie-worksheets/worksheets/credentials.json  
  inflating: genie-worksheets/worksheets/service_account.json  


In [4]:
!mv "genie-worksheets/worksheets/credentials.json" "genie-worksheets/src/worksheets/config/credentials.json"
!mv "genie-worksheets/worksheets/service_account.json" "genie-worksheets/src/worksheets/config/service_account.json"
!mv "genie-worksheets/worksheets/token.json" "genie-worksheets/src/worksheets/config/token.json"

creds1 = os.path.join(root_dir, "src", "worksheets", "config", "credentials.json")
creds2 = os.path.join(root_dir, "src", "worksheets", "config", "service_account.json")
creds3 = os.path.join(root_dir, "src", "worksheets", "config", "token.json")

for cred in [creds1, creds2, creds3]:
  assert os.path.exists(cred), f"Cannot find the credential file: {cred}"

In [5]:
from worksheets.agent.config import agent_api

CELESTE_LEVELS = [
    {
        "level_id": "1",
        "room_id": "1",
        "difficulty": "easy",
        "objective": "reach the chute",
        "mechanics_involved": ["dash", "jump"],
        "hint_tiers": ["Try dashing right", "Climb the wall after dashing", "Use the platform timing"],
        "is_optional": False,
        "provenance_url": "https://celestegame.fandom.com/wiki/Forsaken_City",
        "side": "A"
    }
    # Will eventually be a full database, more will be added later.
]

@agent_api("get_hint", "Get a hint for a Celeste level")
def get_hint(
    level_id: str, 
    room_id: str, 
    hint_tier: int
):
    """
    Get a hint for a specific level.
    
    Args:
        level_id: The level ID
        room_id: The room ID
        hint_tier: Which hint to return (0, 1, or 2)
    """
    # Find the level
    level = next(
        (lvl for lvl in CELESTE_LEVELS 
         if lvl["level_id"] == level_id and lvl["room_id"] == room_id),
        None
    )
    
    if not level:
        return {
            "status": "error",
            "message": f"Level {level_id}, Room {room_id} not found"
        }
    
    # Validate hint_tier
    if hint_tier < 0 or hint_tier >= len(level["hint_tiers"]):
        return {
            "status": "error",
            "message": f"Hint tier {hint_tier} not available. Available: 0-{len(level['hint_tiers'])-1}"
        }
    
    return {
        "status": "success",
        "level_id": level_id,
        "room_id": room_id,
        "objective": level["objective"],
        "hint_tier": hint_tier,
        "hint": level["hint_tiers"][hint_tier]
    }

In [6]:
# Enter the ID of your google sheet here
gsheet_id_default = "1kpbfZkxR288BmRQ7oxaIbGfPrP4xgqDrAcx8OQz6qcg"

Ensure to share with the following account: ``yelpbotvm@mixedinitiative.iam.gserviceaccount.com``

In [7]:
botname = "CelesteGenie"
starting_prompt = "Hello, I am CelesteGenie. I can help you with hints and tips for Celeste levels. Which level and room are you working on?"
description = "CelesteGenie is an assistant that provides hints and tips for players navigating levels in the game Celeste. Users can specify the level and room they are working on, and CelesteGenie will offer tiered hints to help them progress. The assistant is knowledgeable about various mechanics in the game, such as dashing and jumping, and can provide strategic advice based on the player's current situation. CelesteGenie aims to enhance the gaming experience by offering helpful guidance without giving away complete solutions, encouraging players to think critically and improve their skills."

In [8]:
# Helper code to convert the dialogue state to JSON format

from worksheets.utils.annotation import get_agent_action_schemas, get_context_schema
from worksheets.core.dialogue import CurrentDialogueTurn

def convert_to_json(dialogue: list[CurrentDialogueTurn]):
    json_dialogue = []
    for turn in dialogue:
        json_turn = {
            "user": turn.user_utterance,
            "bot": turn.system_response,
            "turn_context": get_context_schema(turn.context),
            "global_context": get_context_schema(turn.global_context),
            "system_action": get_agent_action_schemas(turn.system_action),
            "user_target_sp": turn.user_target_sp,
            "user_target": turn.user_target,
            "user_target_suql": turn.user_target_suql,
        }
        json_dialogue.append(json_turn)
    return json_dialogue

In [9]:
# You need to define the models to use for each componenet of Genie Worksheets.
# You can also load the configurations from a yaml file.

from worksheets.agent.config import Config, AzureModelConfig, OpenAIModelConfig

config = Config(
    semantic_parser = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    response_generator = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    knowledge_parser=OpenAIModelConfig(),
    knowledge_base=OpenAIModelConfig(),
    validate_response= False,

    prompt_log_path = "logs.log",
    conversation_log_path= "conv_log.json",
)

In [10]:
# Initialize your llm by providing it the prompts directory and your `.env` file which contains the secrets

from worksheets.llm.prompts import init_llm

# Use paths relative to current directory
prompts_path = os.path.join(os.getcwd(), "genie-worksheets", "src", "worksheets", "prompts")
env_path = os.path.join(os.getcwd(), ".env")

init_llm(prompts_path, env_path)

In [ ]:
# Main agent builder that generates the agent for you
from worksheets import AgentBuilder, conversation_loop
import json
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="ERROR")

agent_builder = (
    AgentBuilder(
        name=botname,
        description=description,
        starting_prompt=starting_prompt
    )
    .with_gsheet_specification(gsheet_id_default)
)

agent = agent_builder.build(config)


/mnt/c/Users/hallj/GitHub/CS-224V-Project/CS-224V-GameGenie/genie-worksheets/src/worksheets/llm/llm.py:61: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [12]:
for ws in agent.runtime.genie_worksheets:
  print(ws)

RequestHint(level_id: str, room_id: str, hint_tier: Enum['1', '2', '3'], confirm_details: bool)


# INTERACTION WITH AGENT

In [13]:
with agent:
  await conversation_loop(agent, debug=True)

Agent: Hello, I am CelesteGenie. I can help you with hints and tips for Celeste levels. Which level and room are you working on?

request_hint = RequestHint(level_id='', room_id='', hint_tier='', confirm_details=False)
User: a
Agent: > Please provide the ID of the level you need help with.

request_hint_1 = RequestHint(level_id='1')
request_hint = request_hint_1
User: a
Agent: > Please provide the ID of the level you need help with.
User: 1
Agent: > Please provide the ID of the level you need help with.

request_hint_1 = RequestHint(level_id='1', room_id=None, hint_tier=None, confirm_details=False)
User: a
Agent: > Please provide the ID of the level you need help with.
User: 1
Agent: > Please provide the ID of the level you need help with.
User: 1
Agent: > Thank you! You've provided the level ID as **1**. Now, could you please specify the room ID for which you need help?



2025-11-30 11:50:02.324 | ERROR    | worksheets.core.runtime:execute:311 - Error: name 'hint_tier' is not defined
2025-11-30 11:50:02.325 | ERROR    | worksheets.core.runtime:execute:312 - Code: AskField(RequestHint(level_id='1', room_id='1', hint_tier='None', confirm_details=False), hint_tier, 'Please specify the tier of hint you would like (1, 2, or 3).')


request_hint = RequestHint(level_id='1', room_id='1', hint_tier='None', confirm_details=False)
AskField(RequestHint(level_id='1', room_id='1', hint_tier='None', confirm_details=False), hint_tier, 'Please specify the tier of hint you would like (1, 2, or 3).')
User: a
Agent: > Please provide the ID of the level you need help with.
User: 1
Agent: > Please provide the ID of the level you need help with.
User: 1
Agent: > Thank you! You've provided the level ID as **1**. Now, could you please specify the room ID for which you need help?
User: 1
Agent: > You've provided the level ID as **1**. Now, could you please specify the room ID for which you need help?

